# Individual Analysis for the H-D Spectrum Experiment

## Peak-fit method

This template uses a multiple peak-fitting calculation to obtain the peak locations.  It is more sophisticated than the centroid method, but shares some features.

Use this template to carry out the analysis tasks for the HD spectrum experiment.  You may need to consult the documentation for different Python packages.  Also recommended: the [Whirlwind Tour of Python](https://jakevdp.github.io/WhirlwindTourOfPython/) and the [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) both by Jake VanderPlas.

## First, import some packages

This is a good idea at the beginning of your notebook to include the packages that you will need.  We will use the four shown below here.  A brief description:
* `numpy` is the foundational package for Python numerical work. It extends and speeds up array operations beyond standard Python, and it includes almost all math functions that you would need for example `sqrt()` (square root) or `cos()` (cosine).  These would be written in code as `np.sqrt()` or `np.cos()`.
* `scipy` is a huge collection of scientific data analysis functions, routines, physicical constants, etc.  This is the second most used package for scientific work. Documentation is at [SciPy.org](https://docs.scipy.org/doc/scipy/reference/) with the constants subpackage at https://docs.scipy.org/doc/scipy/reference/constants.html.
* `uncertainties` is a very useful small package that simplifies uncertainty propagation and printing out of quantities with uncertainty.  Documentation is at https://pythonhosted.org/uncertainties/
* `matplotlib` is *the* standard plotting package for scientific Python.  We will use a subset called `pyplot` which is modeled after the plotting functions used in MATLAB. The last line below, `%matplotlib inline`, simply forces the plots to appear within the notebook.
* `pandas` is a large data science package.  It's main feature is a set of methods to create and manipulate "Dataframes", which is an enlargement of the idea of an array.  It plays well with NumPy and other packages.  We will use it mainly as a way to read files into data sets in an easy way.

We will also be using the [**LMFit**](https://lmfit.github.io/lmfit-py/) curve fitting package.  This library is very powerful and relatively easy to use to perform complex fitting of functions to data sets, and to also extract meaningful values for statistical uncertainties in the fitting parameters.  It will be introduced and imported later in the template.  Documentation is at https://lmfit.github.io/lmfit-py/

In [ ]:
# Run this cell with Shift-Enter, and wait until the asterisk changes to a number, i.e., [*] becomes [1]
import numpy as np
import scipy.constants as const
import uncertainties as unc
import uncertainties.unumpy as up
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

## Part 1 - Calibration

### Read in the data

It is always a good idea to make a plot of the raw data before trying to manipulate it. Before doing this, you should open a data file with a simple text editor to see what it looks like.  You will see some information at the beginning about th oscilloscope's parameters.

Then use the **Pandas** function `read_csv()` to pull the file into a Pandas Dataframe, like this:

```
Na = pd.read_csv('Na_D_lines.csv', skiprows=15)
```

The variable `skiprows` makes the function skip over the header information in the data file produced by the oscilloscope.

If the last line in the cell is the name of the dataframe, it will print a nice table.

You may obtain the arrays for each column by using the column label, e.g., `Na['CH1']` is the array of the first column.

In [ ]:
# These lines allow the user to enter a data file
# df = pd.read_csv(filename,skiprows=15)  # skiprows skips header information.

# Collect all the data sets into active dataframes.
Na = pd.read_csv('Na_D_lines_run1.csv',skiprows=15)      


In [ ]:
# Just using the label alone causes the dataframe to be displayed.

# The "[:10]" truncates the view to the first 10 rows.

Na[:10]

### Plot the data

Below, I show how. Study the commands, change them, and see what happens.  Hint: study the sections in the [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) on Matplotlib.

Note also the minus sign in front of `Na['CH1']` in the `plt.plot()` function below.  What happens if you remove it?


In [ ]:
plt.figure(figsize=(12,8))
plt.grid()
plt.title('Na Calibration scan')
plt.ylabel(r'Intensity (V)')
plt.xlabel(r'Time (s)')
plt.plot(Na['TIME'],-Na['CH1'],'-',label='Na D-lines')
plt.legend();

### Locate the peak maxima

There are two parts to this.  Part 1 is to smooth the data, part 2 is to locate the maxima of the smoothed data and the width of the peaks located under the maxima.

To do this we will need two functions from SciPy.

In [ ]:
## Import functions from SciPy's collection.  You only need to do an import once in a given session
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks

**Part 1.** First, you want to smooth the y-axis (voltage) data a little bit.  Look closely at the plot, and you will notics a kind of "stairstep" quality, especially near the baseline.  This is the limit of the digitization resolution. Smoothing the data a small amount will remove much of this and make it easier to see the shape of the curve.

One way is to use "Gaussian filtering" to do this.  Gaussian filtering uses a gaussian to convolve with the data set.  The numerical parameter parameter gives the width of the gaussian; larger widths mean more smoothing.

In [ ]:
# Smoothing with a parameter of 5
# This way of doing it adds another column to the dataframe.
Na['CH1_smoothed'] = gaussian_filter1d(Na['CH1'], 5)

Try plotting the smoothed data over the original data.  Then change the `5` in the `gaussian_filter1d()` function to `100` and see what happens.  **Message:** Be careful with smoothing data sets!

In [ ]:
plt.figure(figsize=(12,8))
plt.grid()
plt.title('Na Calibration scan')
plt.ylabel(r'Intensity (V)')
plt.xlabel(r'Time (s)')
plt.plot(Na['TIME'],-Na['CH1'],'-',label='Na D-lines')
plt.plot(Na['TIME'],-Na['CH1_smoothed'],'-',label='Smoothed data')
plt.legend();

**Part 2.** Second, you can use a peak-finding function from SciPy.  This will locate the array indices of peaks in the data sets that satisfy certain criteria.  Using this function takes some care.  You should plot the peak locations and other parameters you are trying to find to make sure it is doing what you want.

I show an example below.

In [ ]:

# assign current arrays to xdata, ydata
ydata = -Na['CH1_smoothed']
xdata = Na['TIME']

# A "min width" keeps small fluctuations near the top from being labeled separate peaks
# Width units are array indices
min_width = 50

# Below does the work.  The height parameter makes the function only look 
# for peaks higher than halfway up the tallest peak.
peaks, pk_props = find_peaks(ydata, width = min_width, height = ydata.max()/2.)

for pk, prop in zip(peaks, pk_props['widths']): 
    print('Peak at {:d} has width {:.1f}'.format(pk, prop))


Show this with a plot.  Note the use of vlines() and hlines()

In [ ]:
plt.figure(figsize=(12,8))
plt.grid()
plt.title('Na Calibration scan')
plt.ylabel(r'Intensity (V)')
plt.xlabel(r'Array index')
plt.plot(ydata,'-',label='Smoothed data')
plt.vlines(x=peaks, ymin=0, ymax=ydata[peaks], color='C2', label='Locations')
plt.hlines(y=pk_props['width_heights'], xmin=peaks-pk_props['widths']/2, 
           xmax=peaks+pk_props['widths']/2, color = 'C1', label='Widths')
plt.legend();

You can now save the peak positions and widths.  These will be used to determine starting values for a peak-fitting routine.

The peak positions and widths are in units of array index.  One needs to convert to the units of the `xdata`.

In [ ]:
xpeaks = np.array([xdata[peaks[0]], xdata[peaks[1]]])
xpeaks

In [ ]:
idx2time = (xdata[peaks[1]]-xdata[peaks[0]])/(peaks[1]-peaks[0])
xwidths = pk_props['widths']*idx2time
xwidths

## Fit the peaks

To fit the data to peaks, make use of the **lmfit** package. It is a useful add-on to the SciPy fitting functions.  This package simplifies fitting data to a variety of standard functions.  See the [Lmfit Documentation](https://lmfit.github.io/lmfit-py/index.html) for a full discussion.  The package is quite powerful, but for basic fitting with common functions, it is very easy to use.  

#### Example: Fitting a line

We'll start with an example fit to a line.  You will use a line fit later in the notebook, but it is simpler than a peak fit, so it makes a good beginning example.

The example shows how to fit data to a line, obtain the fit parameters along with uncertainties, and then plot the data and fit. Execute the cells and study how it works. 

The first cell is just example data. (They come from a calibration problem in physics 331, optics lab)  You will modify the function calls when you use it on your own results later

In [ ]:
# Calibration Data
# First column is wavelength (nm), second is carriage poisition (cm)
#
Cal_data = np.array([
    [643.85, 41.43],
    [579.07, 37.24],
    [576.96, 37.11],
    [546.08, 35.10],
    [508.58, 32.68],
    [479.99, 30.83],
    [467.81, 30.04],
    [435.83, 27.96],
    [404.66, 25.98]])

# Array slicing separates x (position) and y (wavelength)
# Goal of calibration is to be able to feed in a position and obtain a wavelength
wavelength = Cal_data[:,0]
position = Cal_data[:,1]

Here is the fitting example.  Note the very first command to import a fitting "Model" called `LinearModel`.

A "Model" is an LMFit object that contains information about the fitting function and methods to help set up and manipulate the fitting parameters.  Notice what each line does, and how the return value of the function call is used in subsequent calls.

In [ ]:
# imports a linear fitting model from lmfit
# You only need to execute this line once!
from lmfit.models import LinearModel

# create an instance of the model
line = LinearModel()

# One must have a guess of the parameters. The guess() method works with most of the standard
# lmfit models
param_guess = line.guess(wavelength, x=position)

# The line below executes the fitting process.  The results are returned to "line_fit"
line_fit = line.fit(wavelength, param_guess, x=position)

# This prints the results in an easy to read form
print(line_fit.fit_report())

# The parameters and uncertainties are accessible as follows, for example:
print('\nSlope = ',line_fit.params['slope'].value,'+/-',line_fit.params['slope'].stderr)

#Then you can plot the results quickly just to see how it looks using the plot() method
line_fit.plot();

## Make the peak-fitting model

Now we expand the example to a more complicated problem: a fit of two Gaussian peaks and a constant background.

The Gaussian peak has the form
$$f(x; A,\mu,\sigma) = \frac{A}{\sigma \sqrt{2\pi}}e^{[(x-\mu)^2/2\sigma^2]}\;.$$ 
The parameters are `amplitude`, `center`, and `sigma`. In addition, parameters `fwhm` and `height` are included as constraints to report full width at half maximum and maximum peak height, respectively.

Note that the `height` of the peak at the center ($x=\mu$) is equal to $\frac{A}{\sigma\sqrt{2\pi}}$ and that the full-width at half-maximum is about 2.3548$\sigma$.

Thus, to estimate the amplitude $A$, estimate the height $h$ and FWHM from a graph, and calculate $A = h*$FWHM.

The contant background sets the height of the data set "floor." 

In [ ]:
# Import a gaussian peak and second order polynomial for background

from lmfit.models import ConstantModel, GaussianModel

# create an instance of the model
# Note the use of prefixes
model = ConstantModel() + GaussianModel(prefix='p1_') + GaussianModel(prefix='p2_')

# Necessary step below.  This replaces the guess() method used earlier.  The guess() method 
# does not work for composite models.
params = model.make_params()

# Notice how the prefixes are attached to the parameter names.
print('parameter names: {}'.format(model.param_names))

### Setting the parameters

For a composite fit, when you cannot use `guess()`, you need to set the values of the parameters by hand. Information has already been found to do this from the above peak-finding steps.

Study the lines below, and note how the values of each parameter is set.

Also notice the use of the `vary` property for each fitting parameter.  If you set `vary =  False`, then that parameter will not be adjusted during the fit.  

This is one way to make a line fit go through zero. For example:

```param_guess['intercept'].set(value=0, vary=False)``` 

placed before the call `line_fit = line.fit(wavelength, param_guess, x=position)` in the line fit example above would force the line through zero and find the best fit slope under that constraint.

In [ ]:
# Peak 1 starting values

p1_loc = xpeaks[0]
p1_sig = xwidths[0]/2.3548
p1_amp = ydata[peaks[0]]*xwidths[0]

# Peak 2 starting values

p2_loc = xpeaks[1]
p2_sig = xwidths[1]/2.3548
p2_amp = ydata[peaks[1]]*xwidths[0]

# Build the parameters

params['p1_center'].set(value=p1_loc, vary=True)
params['p1_amplitude'].set(value=p1_amp, vary=True)
params['p1_sigma'].set(value=p1_sig, vary=True)

params['p2_center'].set(value=p2_loc, vary=True)
params['p2_amplitude'].set(value=p2_amp, vary=True)
params['p2_sigma'].set(value=p2_sig, vary=True)

params['c'].set(value = 0.01, vary=True)

params.pretty_print(columns=['value','vary'])

After setting the parameters, the rest of the fitting process is the same as for a line.

In [ ]:
model_fit = model.fit(ydata, params, x=xdata)

print(model_fit.fit_report(show_correl=False))

myfig=plt.figure(figsize=(10,10))
model_fit.plot(fig=myfig);

Save the peak locations and uncertainty into uncertainty objects `ufloat`.  These are in the parameters `p1_center` and `p2_center`.  Notice how the code below searches out the `_center` parameters using a string search function.

Calculations with uncertainty objects will automatically propagate the uncertainty.

In [ ]:
Peak_loc = []

for parm in model_fit.params:
    if parm.endswith('center'):
        # make an uncertainty object
        this_center = unc.ufloat(model_fit.params[parm].value, model_fit.params[parm].stderr)
        # add it to the Python list
        Peak_loc.append(this_center)
        # also print it
        print('Peak at {:.2uP} s'.format(this_center))

# Save the list to a new variable 
Na_loc = Peak_loc.copy()

Next, calculate the calibration constant with its uncertainty.  This is easy if you are using the uncertainty object.

The calibration constant $K_\text{Na}$ is simply

$$ K_\text{Na} = \frac{\lambda_{\text{D}_1}-\lambda_{\text{D}_2}}{t_{\text{D}_1}-t_{\text{D}_2}}$$

By definition, $D_1$ has a longer wavelength than $D_2$.

In [ ]:
D1 = 5895.92 # Angstroms
D2 = 5889.95 # Angstroms

Na_cal = (D1-D2)/(Na_loc[1]-Na_loc[0])

print('Calibration constant from peak-fit method: {:.2uP} A/s'.format(Na_cal))

## Part 2 - Data reduction

### Read in and plot all of the data

Before doing any calculations, you shoul always plot and look hard at your data.

Repeat the steps at the beginning to feed each data set into a Pandas dataframe.

In [ ]:
## Collect all the data sets into active dataframes. 
## Follow the method used for the Na lines

## Recommended names  for te dataframes 'Alpha', 'Beta', 'Gamma', etc.

Now plot all on one graph.  Note use of "-" to flip all data sets to positive

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(Na['TIME'],-Na['CH1'],'-',label='Na D-lines')
plt.plot(Alpha['TIME'],-Alpha['CH1'],'-',label=r'$\alpha$')

# ....
# add the rest here

plt.xlabel(r'Time (s)')
plt.ylabel(r'Ch 1 (V)')
plt.legend()
plt.grid();

Hard to read.  Let's offset the plots and rescale them.  Note use of multiplication factors

In [ ]:
## Cut and paste above, and then manipulate arrays within the plot() calls


### Process each data file

At this point, you want to apply the same basic steps to each H-D spectrum files, namely:

* Smooth it.
* Locate the peak positions and widths.
* Use this information to feed the starting values for the fitting model.
* Extract the peak centers and uncertainty from the fit results.
* Calculate $\Delta t_\text{HD}$ for that data set, and save it.

Before doing these steps, convert the code blocks used to carry out these tasks above to functions.  This will make your code reuse less messy and easier to run.

In [ ]:
## Function to locate the peaks, plot them, and return the important data

def find_and_plot_peaks(ydata, title='Data', makeplot=True):
    '''
    Function locates peaks using SciPy.signal find_peaks() and plots
    the results (optionally).  Returns two arrays: 'peaks' which holds
    the index of each peak maximum and 'widths' which holds the FWHM of
    each peak.
    '''
    # You fill this in here  
        
    return peaks, pk_props['widths']


In [ ]:
## Function to obtain starting values for fit
def guess_start_parms(xdata, ydata, peaks, widths, params):
    '''
    Calculates starting values for the peak fitting routine.  
    Returns params object
    '''
    # You fill in the rest
    
    return params


In [ ]:
def calculate_peak_locs(xdata, ydata, params):
    '''
    Fit data to two gaussian peaks and extract peak positions.
    Plots result and extracts centers with uncertainty.
    Returns a list of uncertainty objects
    '''
    
    return Peak_loc


In [ ]:
## Test the above.  Recommend commenting out later lines and examining the return values at each step

Na_pks, Na_widths = find_and_plot_peaks(ydata, title='Na Lines', makeplot=True)
Na_start_params = guess_start_parms(xdata, ydata, Na_pks, Na_widths, params)
Na_test_loc = calculate_peak_locs(xdata, ydata, Na_start_params)


### Apply the functions

Work on Alpha first. I show this as an example.  Reuse `xdata`, `ydata`, `peaks`, `widths` and `start_params` to save typing.  **But be careful:** if you execute cells out of sequence, you will get peculiar results!!

In [ ]:
Alpha['CH1_smoothed'] = gaussian_filter1d(Alpha['CH1'], 5)

xdata = Alpha['TIME']
ydata = -Alpha['CH1_smoothed']

peaks, widths = find_and_plot_peaks(ydata, title='Alpha Line', makeplot=True)

Next step: Fit and grab peaks.

Results saved to `Alpha_loc`.  Will repeat for `Beta_loc`, `Gamma_loc`, etc.

In [ ]:
start_params = guess_start_parms(xdata, ydata, peaks, widths, params)

Alpha_loc = calculate_peak_locs(xdata, ydata, start_params)

### Repeat for other lines.  

Beta next.

In [ ]:
Beta['CH1_smoothed'] = gaussian_filter1d(Beta['CH1'], 5)

xdata = Beta['TIME']
ydata = -Beta['CH1_smoothed']

## Etc... 

In [ ]:
# second and third calls

Gamma next.

In [ ]:
# You what to do now. 

Delta next.

Finally, Epsilon lines (2 sets).

Epsilon 2.

## Save the time differences

One easy way is to make a simple Python list:

In [ ]:
D_T = [Alpha_loc[1]-Alpha_loc[0],
      Beta_loc[1]-Beta_loc[0],
      Gamma_loc[1]-Gamma_loc[0],
      Delta_loc[1]-Delta_loc[0],
      Eps1_loc[1]-Eps1_loc[0],
      Eps2_loc[1]-Eps2_loc[0]]
D_T

A fancier way is to build a Pandas Series.  I will also average the two epsilon runs.

In [ ]:
Delta_T = pd.Series({'Alpha': Alpha_loc[1]-Alpha_loc[0],
                   'Beta': Beta_loc[1]-Beta_loc[0],
                   'Gamma': Gamma_loc[1]-Gamma_loc[0],
                   'Delta': Delta_loc[1]-Delta_loc[0],
                   'Epsilon': (Eps1_loc[1]+Eps2_loc[1]-Eps1_loc[0]-Eps2_loc[0])/2.0})
Delta_T

Then include the known hydrogen wavelengths $\lambda_\text{H}$.  A table can be found in Haken & Wolf, p.98

In [ ]:
lambda_H = pd.Series({'Alpha': 6562.79,
                   'Beta': 4863.33,
                   'Gamma': 4340.46,
                   'Delta': 4101.73,
                   'Epsilon': 3970.07})
lambda_H

Build into a dataframe.

In [ ]:
HD_results = pd.DataFrame({'lambda_H (A)':lambda_H, 'Delta-t (s)': Delta_T})
HD_results

In [ ]:
HD_results['Delta-lambda (A)'] = HD_results['Delta-t (s)']*Na_cal
HD_results

## Make a plot

Plot the results of $\Delta t_\text{HD}$ versus $\lambda_\text{H}$ with the uncertainty as errorbars.  Include the origin in the plot.

In [ ]:
plt.figure(figsize=(12,8))
plt.grid()
plt.title('HD spectrum results')
plt.ylim(0,9.8)
plt.xlim(0,6800.0)
plt.ylabel(r'Time difference between H,D peaks $\Delta t_{\rm HD}$ (s)')
plt.xlabel(r'Hydrogen wavelength (A)')
plt.errorbar(HD_results['lambda_H (A)'],up.nominal_values(HD_results['Delta-t (s)']),fmt='o',
             yerr=up.std_devs(HD_results['Delta-t (s)']),label='Spectrum data');
plt.legend();



## Fit to a line

Fit these results to a line that goes through zero.  Go back to the line fitting example, and copy the relevant code.


In [ ]:
## Put the data into simple arrays: xdata, ydata, yuncert

xdata = HD_results['lambda_H (A)']
ydata = up.nominal_values(HD_results['Delta-t (s)'])
yuncert = up.std_devs(HD_results['Delta-t (s)'])

## Then use the earlier code starting with "param_guess = line.guess(wavelength, x=position)"
## and change the variable assignments appropriately to make your fit.

## Make a nice plot

Replot the above data-plot but include the fitline so that you can see the line go through zero.

In [ ]:
## Code below makes a fitline

xfit = np.linspace(0,6600,10)
yfit = line_fit.eval(line_fit.params, x=xfit)

## Cut and paste the earlier plot, and include the fitline.


In [ ]:
## Save the slope from the fit.
HD_slope = unc.ufloat(line_fit.params['slope'].value,line_fit.params['slope'].stderr)

## Then use it in a calculation to obtain the mass ratio and uncertainty, and print it out

## You write code to calculate the mass ratio.

print('Calculated Mass ratio of hydrogen/deuterium = {:.1uP}'.format(mass_ratio))